# Pré-traitement du corpus(enlever les ponctuation et lemmatization)

In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string


nltk.download('punkt')
nltk.download('wordnet')


df = pd.read_csv('/Users/shailynnxie/Documents/M2/Logicielle/ubuntu_sample.tsv', sep='\t', header=None)


lemmatizer = WordNetLemmatizer()

def preprocess_text(text):

    text = text.translate(str.maketrans('', '', string.punctuation))

    tokens = word_tokenize(text)

    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(lemmatized_words)

df.iloc[:, 1] = df.iloc[:, 1].apply(preprocess_text)
df.iloc[:, 2] = df.iloc[:, 2].apply(preprocess_text)

df.to_csv('/Users/shailynnxie/Documents/M2/Logicielle/processed_file.csv', index=False)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shailynnxie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shailynnxie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Lemmatization du librairie NLTK n'est pas parfait

Il ne peut pas lemmatizer le temps du verbe

In [51]:
print(lemmatizer.lemmatize('did'))
print(lemmatizer.lemmatize('doing'))


did
doing


# TF-IDF avec 2 gram

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


df = pd.read_csv('/Users/shailynnxie/Documents/M2/Logicielle/processed_file.csv')

# obtenir question et answer
texts_2 = df.iloc[:, 1].values.tolist()
texts_3 = df.iloc[:, 2].values.tolist()

# Créez un vectoriseur TF-IDF et définissez ngram_range sur (1, 2) pour utiliser uni-gramme et bi-gramme
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))

#vectorier les answers
tfidf_matrix = tfidf_vectorizer.fit_transform(texts_3)


matches = []

# match les answers avec chaque question
for text in texts_2:

    text_tfidf = tfidf_vectorizer.transform([text])
    similarity_scores = cosine_similarity(text_tfidf, tfidf_matrix)
    best_match_index = similarity_scores.argmax()
    matches.append(texts_3[best_match_index])

df['best_match'] = matches

df.to_csv('/Users/shailynnxie/Documents/M2/Logicielle/TFIDF 2gram.csv', index=False)


In [57]:
column3 = df.iloc[:, 2]
column4 = df.iloc[:, 3]

    
consistent_count = 0


for value3, value4 in zip(column3, column4):
        if value3 == value4:
            consistent_count += 1


total_rows = len(df)
consistency_rate = consistent_count / total_rows
print(f" Aucuracy Rate: {consistency_rate:.2%}")

 Aucuracy Rate: 63.10%


### le résultat est un peu moins bien que l'autre fois(sans prétraitement (65.90%))

# SIF similarité


In [ ]:
import numpy as np
from collections import Counter
import itertools
from sklearn.metrics import pairwise_distances
import gensim.downloader as api

#loading un model pour entrainement
glove_vectors = api.load('glove-twitter-200')

# calculer SIF feature et vectoriser
def map_word_frequency(document):
    return Counter(itertools.chain(*document))

def get_sif_feature_vectors(texts, word_emb_model):
    sif_weighted_embeddings = []
    for text in texts:

        tokens = text.split()

        tokens = [token for token in tokens if token in word_emb_model]

        word_counts = map_word_frequency([tokens])

        sentence_vec = np.zeros(word_emb_model.vector_size)
        for word in tokens:
            # calcul SIF-weighted 
            alpha = 0.001
            weight = alpha / (alpha + word_counts[word])
            sentence_vec += weight * word_emb_model[word]
        # normalization
        sentence_vec /= len(tokens)
        
        sif_weighted_embeddings.append(sentence_vec)
    return sif_weighted_embeddings

df = pd.read_csv('/Users/shailynnxie/Documents/M2/Logicielle/processed_file.csv')
df.fillna('', inplace=True)


texts_2 = df.iloc[:, 1].values.tolist()
texts_3 = df.iloc[:, 2].values.tolist()

sif_vectors_2 = get_sif_feature_vectors(texts_2, glove_vectors)
sif_vectors_3 = get_sif_feature_vectors(texts_3, glove_vectors)

# transférer les vecter aux matrix NumPy
sif_vectors_2 = np.array(sif_vectors_2)
sif_vectors_3 = np.array(sif_vectors_3)
for dist_name in ["jaccard", "euclidean","manhattan", "cosine"]:
    distances = pairwise_distances(sif_vectors_2, sif_vectors_3,metric=dist_name)

    min_distances_indices = np.argmin(distances, axis=1)
    matched_answers = df.iloc[min_distances_indices, 2].values
    similarity_scores = 1 - np.min(distances, axis=1)

    
    df['best_match'] = matched_answers
    df['similarity_score'] = similarity_scores

    df.to_csv('/Users/shailynnxie/Documents/M2/Logicielle/SIF_similarity_%s.csv'% dist_name, index=False)

#print(distances)
#print(distances.shape)
#print(matched_answers)

In [44]:
for dist_name in ["jaccard", "euclidean","manhattan", "cosine"]:


    df = pd.read_csv("/Users/shailynnxie/Documents/M2/Logicielle/SIF_similarity_%s.csv" % dist_name)

    # comparer si l'étément dans la colone 2 et 3 sont identiques
    column3 = df.iloc[:, 2]
    column4 = df.iloc[:, 3]

    
    consistent_count = 0


    for value3, value4 in zip(column3, column4):
        if value3 == value4:
            consistent_count += 1


    total_rows = len(df)
    consistency_rate = consistent_count / total_rows

    print(dist_name,f" Aucuracy Rate: {consistency_rate:.2%}")

jaccard  Aucuracy Rate: 0.10%
euclidean  Aucuracy Rate: 13.30%
manhattan  Aucuracy Rate: 17.50%
cosine  Aucuracy Rate: 23.20%


### Similarité euclidean et  manhattan sont améliorer comparant dernière fois, mais jaccard et cosine sont beaucoup dégradés


In [20]:
df = pd.read_csv("/Users/shailynnxie/Documents/M2/Logicielle/matched_results.csv" )

# comparer si l'étément dans la colone 2 et 3 sont identiques
column3 = df.iloc[:, 1]
column4 = df.iloc[:, 2]


consistent_count = 0


for value3, value4 in zip(column3, column4):
    if value3 == value4:
        consistent_count += 1


total_rows = len(df)
consistency_rate = consistent_count / total_rows

print(f" Aucuracy Rate: {consistency_rate:.2%}")

 Aucuracy Rate: 27.80%


# L'algorithme Hongrois

## Conbinaison avec TF-IDF pondération

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from scipy.optimize import linear_sum_assignment
# Charger le fichier CSV prétraité
df = pd.read_csv('/Users/shailynnxie/Documents/M2/Logicielle/processed_file.csv')
df.fillna('', inplace=True)# Remplacer les valeurs manquantes par des chaînes vides

V = TfidfVectorizer()# Initialisation du vectorisateur TF-IDF

# Vectorisation des réponses (colonne 3) avec TF-IDF
answers_vectors = V.fit_transform(df.iloc[:, 2]).toarray()

# Calcul de la matrice des distances entre les questions (colonne 2) et les réponses vectorisées
# Utilisation de la distance cosinus
distances_matrix = pairwise_distances(V.transform(df.iloc[:, 1]).toarray(), answers_vectors, metric='cosine')

# Application de l'algorithme hongrois pour trouver le meilleur appariement entre questions et réponses

row_indices, col_indices = linear_sum_assignment(distances_matrix)
# Extraction des réponses correspondantes aux indices trouvés par l'algorithme

matched_answers = df.iloc[col_indices, 2].values
actual_answers = df.iloc[:, 2].values

# calculer exactitude
correct_matches = (actual_answers == matched_answers).sum()
accuracy_rate = correct_matches / len(df)

print(f"Accuracy Rate: {accuracy_rate:.2%}")

# Sauvegarde des résultats dans un nouveau fichier CSV pour voir les q&a qui sont bien matche
results_df.to_csv("C:/Users/64271/Desktop/gael/matching_results_with_accuracy.csv", index=False)

Accuracy Rate: 76.70%


In [ ]:
# Charger le fichier CSV prétraité
df = pd.read_csv('C:/Users/64271/Desktop/gael/processed_file.csv')
df.fillna('', inplace=True)  # Remplacer les valeurs manquantes par des chaînes vides

# Initialisation du vectorisateur TF-IDF
V = TfidfVectorizer()

# Vectorisation des réponses (colonne 3) avec TF-IDF
answers_vectors = V.fit_transform(df.iloc[:, 2]).toarray()

# Calcul de la matrice des distances entre les questions (colonne 2) et les réponses vectorisées
# Utilisation de la distance cosinus
distances_matrix = pairwise_distances(V.transform(df.iloc[:, 1]).toarray(), answers_vectors, metric='cosine')

# Application de l'algorithme hongrois pour trouver le meilleur appariement entre questions et réponses
row_indices, col_indices = linear_sum_assignment(distances_matrix)

# Extraction des réponses correspondantes aux indices trouvés par l'algorithme
matched_answers = df.iloc[col_indices, 2].values
actual_answers = df.iloc[:, 2].values  # Les réponses actuelles (supposées être dans la colonne 3)

# Calcul du taux de précision en comparant les réponses actuelles aux réponses appariées
correct_matches = (actual_answers == matched_answers).sum()
accuracy_rate = correct_matches / len(df)

# Création d'un nouveau DataFrame pour stocker les questions, les réponses actuelles, les réponses appariées
# et si l'appariement est correct
results_df = pd.DataFrame({
    'Question': df.iloc[:, 1],  #la colonne 2 contient les questions
    'Actual Answer': actual_answers,  # Réponses actuelles
    'Matched Answer': matched_answers,  # Réponses appariées par l'algorithme
    'Is Correct': actual_answers == matched_answers  # Indique si l'appariement est correct
})

# Calcul et affichage du taux de précision
accuracy_rate = results_df['Is Correct'].mean()
print(f"Taux de Précision: {accuracy_rate:.2%}")

# Sauvegarde des résultats dans un nouveau fichier CSV pour voir les q&a qui sont bien matche
results_df.to_csv("C:/Users/64271/Desktop/gael/matching_results_with_accuracy.csv", index=False)

# Affichage du chemin du fichier pour connaître l'emplacement du fichier sauvegardé
"C:/Users/64271/Desktop/gael/matching_results_with_accuracy.csv"

## Conbinaison avec SIF

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

texts_2 = df.iloc[:, 1].values.tolist()
texts_3 = df.iloc[:, 2].values.tolist()

sif_vectors_2 = get_sif_feature_vectors(texts_2, glove_vectors)
sif_vectors_3 = get_sif_feature_vectors(texts_3, glove_vectors)

# transférer les vecter aux matrix NumPy
sif_vectors_2 = np.array(sif_vectors_2)
sif_vectors_3 = np.array(sif_vectors_3)
# Calculer la distance entre les vecteurs SIF des textes des colonnes 2 et 3
distances = cosine_similarity(sif_vectors_2, sif_vectors_3)

# Utiliser la méthode Hungarian (ou de Kuhn-Munkres) pour trouver les meilleures correspondances
row_ind, col_ind = linear_sum_assignment(-distances)  # Nous utilisons -distances car linear_sum_assignment trouve la correspondance minimale, mais nous voulons maximiser la similarité

# Récupérer les meilleures correspondances et les scores de similarité correspondants
matched_answers = [texts_3[j] for j in col_ind]
similarity_scores = [distances[i, j] for i, j in zip(row_ind, col_ind)]



# Si nécessaire, enregistrer les résultats dans un DataFrame
matched_df = pd.DataFrame({"ID":  df.iloc[:, 0].values.tolist(),'Question': texts_2,'Answer': texts_3, 'Matched_Text_3': matched_answers, 'Similarity_Score': similarity_scores})
matched_df.to_csv('Hangarian sif.csv', index=False)


In [26]:
column3 = matched_df.iloc[:, 1]
column4 = matched_df.iloc[:, 2]


consistent_count = 0


for value3, value4 in zip(column3, column4):
    if value3 == value4:
        consistent_count += 1


total_rows = len(df)
consistency_rate = consistent_count / total_rows

print(f" Aucuracy Rate: {consistency_rate:.2%}")

 Aucuracy Rate: 0.30%


### Selon les résultats, le résultat obtenu en utilisant le vecteur pondéré TFIDF plus l'algorithme hongrois est le meilleur. L‘algorithme hongrois améliore les matchs comparant utilise les calculs de distance.